<a href="https://colab.research.google.com/github/ArifAygun/CustomerEye/blob/main/Company_Reviews(3).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#### IMPORT LIBRARIES AND DATASET

In [41]:
!pip install pycountry

In [42]:
import re
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
import matplotlib.dates as mdates
import pycountry

import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [43]:
from google.colab import drive
import pandas as pd

# Mount Google Drive
drive.mount('/content/drive/')

# Change directory to the analysis folder
%cd /content/drive/My Drive/Analysis_1/

# Read CSV files into DataFrames
df1 = pd.read_csv('Freedom_Debt_Relief.csv')
df2 = pd.read_csv('Millennium_Trust.csv')
df3 = pd.read_csv('Advance_America.csv')

print(df1.info(), df1.head())
print(df2.info(), df2.head())
print(df3.info(), df3.head())

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).
/content/drive/My Drive/Analysis_1
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39078 entries, 0 to 39077
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Reviewer Name    39068 non-null  object
 1   Reviews Count    39078 non-null  object
 2   Country Code     39078 non-null  object
 3   Experience Date  39078 non-null  object
 4   Rating           39078 non-null  int64 
 5   Review Date      39078 non-null  object
 6   Review Title     39074 non-null  object
 7   Review Text      34241 non-null  object
 8   Reply Date       37731 non-null  object
 9   Reply Text       37731 non-null  object
dtypes: int64(1), object(9)
memory usage: 3.0+ MB
None     Reviewer Name Reviews Count Country Code    Experience Date  Rating  \
0  Andrea Sciotti      2reviews           US  December 

### DATA PREPROCESSING

In [44]:
import pandas as pd

# Define a function to calculate response time
def calculate_response_time(df, reply_text):
    # Convert 'Reviews Count' to integers
    df['Reviews Count'] = df['Reviews Count'].str.extract('(\d+)').astype(int)

    # Replace the specified text in 'Reply Date' column
    df['Reply Date'] = df['Reply Date'].str.replace(reply_text, '').str.strip()

    # Convert 'Experience Date', 'Review Date', and 'Reply Date' to datetime
    date_columns = ['Experience Date', 'Review Date', 'Reply Date']
    df[date_columns] = df[date_columns].apply(pd.to_datetime, errors='coerce')

    # Drop rows with NaN values in 'Experience Date', 'Review Date', or 'Reply Date'
    df = df.dropna(subset=date_columns)

    # Create a copy to avoid SettingWithCopyWarning
    df = df.copy()

    # Concatenate 'Review Title' and 'Review Text' into a new 'Reviews' column
    df.loc[:, 'Reviews'] = df['Review Title'].astype(str) + ' ' + df['Review Text'].astype(str)

    # Rename 'Reply Text' column to 'Replies'
    df = df.rename(columns={'Reply Text': 'Replies'})

    # Drop the original 'Review Title' and 'Review Text' columns
    df = df.drop(['Reviewer Name', 'Reviews Count', 'Review Title', 'Review Text'], axis=1)

    # Calculate response time between 'Experience Date' and 'Review Date' in days
    df['Exp to Review'] = (df['Review Date'] - df['Experience Date']).dt.total_seconds() / 86400

    # Calculate response time between 'Review Date' and 'Reply Date' in days
    df['Review to Reply'] = (df['Reply Date'] - df['Review Date']).dt.total_seconds() / 86400

    # Round the values to the nearest integer and convert to int
    df[['Exp to Review', 'Review to Reply']] = df[['Exp to Review', 'Review to Reply']].round(0).astype(int)

  # Rearrange the columns
    df = df[['Experience Date', 'Review Date', 'Reply Date','Exp to Review',
             'Review to Reply', 'Country Code', 'Rating', 'Reviews', 'Replies']]

    return df

# Calculate response time for each DataFrame
df1 = calculate_response_time(df1, 'Reply from Freedom Debt Relief')
df2 = calculate_response_time(df2, 'Reply from Millennium Trust')
df3 = calculate_response_time(df3, 'Reply from Advance America')

# Print the updated DataFrame information
pd.set_option('display.max_columns', None)

print(df1.info())
print(df2.info())
print(df3.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 36713 entries, 0 to 39077
Data columns (total 9 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   Experience Date  36713 non-null  datetime64[ns]
 1   Review Date      36713 non-null  datetime64[ns]
 2   Reply Date       36713 non-null  datetime64[ns]
 3   Exp to Review    36713 non-null  int64         
 4   Review to Reply  36713 non-null  int64         
 5   Country Code     36713 non-null  object        
 6   Rating           36713 non-null  int64         
 7   Reviews          36713 non-null  object        
 8   Replies          36713 non-null  object        
dtypes: datetime64[ns](3), int64(3), object(3)
memory usage: 2.8+ MB
None
<class 'pandas.core.frame.DataFrame'>
Int64Index: 29374 entries, 235 to 39327
Data columns (total 9 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   Experience Dat

In [36]:
df3.sample(20)

,Experience Date,Review Date,Reply Date,Exp to Review,Review to Reply,Country Code,Rating,Reviews,Replies
34975,2020-11-07,2020-11-07,2020-11-07,0,0,US,5,Excellent customer service Excellent customer ...,"Your business is important to us, thank you fo..."
34756,2020-11-18,2020-11-18,2020-11-18,0,0,US,5,Very easy transactions and very… Very easy tra...,Thank you for your response and comments regar...
11387,2023-04-21,2023-04-21,2023-04-21,0,0,US,5,Super great gals Super great gals! Treat their...,Thank you for your positive review. At Advance...
35626,2020-10-21,2020-10-21,2020-10-22,0,1,US,5,My office visit to Advance America I talked wi...,"Your business is important to us, thank you fo..."
35926,2020-10-13,2020-10-13,2020-10-15,0,2,US,5,A very pleasant first experience. nan,"Your business is important to us, thank you fo..."
4500,2023-09-16,2023-09-19,2023-09-19,3,0,US,5,in and out quickly in and out quickly. knowle...,Thank you for your positive review. At Advance...
35423,2020-10-28,2020-10-28,2020-10-28,0,0,US,5,Professional friendly personal Professional fr...,Thank you for your response and comments regar...
8915,2023-06-16,2023-06-19,2023-06-20,3,1,US,4,Friendly service nan,Thank you for your positive review. At Advance...
15400,2022-11-07,2022-11-19,2022-11-19,12,0,US,5,Payments you can live with Complete workout ho...,Thank you for your positive review. At Advance...
15175,2022-11-26,2022-11-26,2022-11-26,0,0,US,5,Was immediately greeted with a warm… Was immed...,Thank you for your positive review. At Advance...


In [4]:
# Define a function to perform the required modifications
def modify_dataframe(df, reply_text):
    # Convert 'Reviews Count' to integers
    df['Reviews Count'] = df['Reviews Count'].str.extract('(\d+)').astype(int)

    # Replace the specified text in 'Reply Date' column
    df['Reply Date'] = df['Reply Date'].str.replace(reply_text, '').str.strip()

    # Convert 'Experience Date', 'Review Date', and 'Reply Date' to datetime
    date_columns = ['Experience Date', 'Review Date', 'Reply Date']
    df[date_columns] = df[date_columns].apply(pd.to_datetime, errors='coerce')

    # Drop rows with NaN values in 'Experience Date', 'Review Date', or 'Reply Date'
    df = df.dropna(subset=date_columns)

    # Create a copy to avoid SettingWithCopyWarning
    df = df.copy()

    # Concatenate 'Review Title' and 'Review Text' into a new 'Reviews' column
    df.loc[:, 'Reviews'] = df['Review Title'].astype(str) + ' ' + df['Review Text'].astype(str)

    # Rename 'Reply Text' column to 'Replies'
    df = df.rename(columns={'Reply Text': 'Replies'})

    # Drop the original 'Review Title' and 'Review Text' columns
    df = df.drop(['Reviewer Name', 'Reviews Count', 'Review Title', 'Review Text'], axis=1)

    # Rearrange the columns
    df = df[['Experience Date', 'Review Date', 'Reply Date', 'Country Code', 'Rating', 'Reviews', 'Replies']]

    return df

# Read CSV files into DataFrames
df1 = pd.read_csv('Freedom_Debt_Relief.csv')
df2 = pd.read_csv('Millennium_Trust.csv')
df3 = pd.read_csv('Advance_America.csv')

# Apply modifications for each DataFrame
df1_modified = modify_dataframe(df1, 'Reply from Freedom Debt Relief')
df2_modified = modify_dataframe(df2, 'Reply from Millennium Trust')
df3_modified = modify_dataframe(df3, 'Reply from Advance America')

# Print the updated DataFrame information
print(df1_modified.info())
print(df2_modified.info())
print(df3_modified.info())


<class 'pandas.core.frame.DataFrame'>
Int64Index: 36713 entries, 0 to 39077
Data columns (total 7 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   Experience Date  36713 non-null  datetime64[ns]
 1   Review Date      36713 non-null  datetime64[ns]
 2   Reply Date       36713 non-null  datetime64[ns]
 3   Country Code     36713 non-null  object        
 4   Rating           36713 non-null  int64         
 5   Reviews          36713 non-null  object        
 6   Replies          36713 non-null  object        
dtypes: datetime64[ns](3), int64(1), object(3)
memory usage: 2.2+ MB
None
<class 'pandas.core.frame.DataFrame'>
Int64Index: 29374 entries, 235 to 39327
Data columns (total 7 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   Experience Date  29374 non-null  datetime64[ns]
 1   Review Date      29374 non-null  datetime64[ns]
 2   Reply Date    

In [ ]:
import pandas as pd

# Define a function to calculate response time
def calculate_response_time(df):
    # Calculate response time between 'Experience Date' and 'Review Date'
    df['Response Time (Exp to Review)'] = df['Review Date'] - df['Experience Date']

    # Calculate response time between 'Review Date' and 'Reply Date'
    df['Response Time (Review to Reply)'] = df['Reply Date'] - df['Review Date']

    return df

# Assuming df1_modified, df2_modified, and df3_modified are your modified DataFrames
df1_modified = calculate_response_time(df1_modified)
df2_modified = calculate_response_time(df2_modified)
df3_modified = calculate_response_time(df3_modified)

# Print the updated DataFrame information
print(df1_modified.info())
print(df2_modified.info())
print(df3_modified.info())


In [ ]:
df.sample(20)

,Experience Date,Review Date,Reply Date,Country Code,Rating,Reviews,Replies
33240,2018-04-19,2018-04-19,2018-04-23,US,3,Too early to tell Too early to tell. Ordinatio...,Thank you for your honest feedback. We look fo...
17453,2020-03-30,2020-03-30,2020-04-01,US,5,"Didn't have to wait Didn't have to wait, great...",Terrific feedback! Thank you for taking the ti...
547,2019-11-07,2019-11-07,2019-11-07,US,4,I would hate to speak out prematurely I would ...,"Robert, Rest assured you are in good hands and..."
13852,2017-11-29,2017-11-29,2017-12-01,US,4,Looking forward for a bright future nan,Thank you for your review and thank you for ch...
38067,2020-11-26,2020-11-26,2020-11-28,US,5,Already half way to debt free nan,"Thank you, Miguel, for your amazing feedback! ..."
16597,2020-05-13,2020-05-13,2020-05-13,US,5,Made us feel secure about our future Made us f...,"It is rewarding to hear such great feedback, C..."
1529,2019-09-26,2019-09-26,2019-09-28,US,5,I like Freedom Debt Relief I like Freedom Debt...,"Thank you, Larry, for your stunning feedback a..."
22268,2019-01-22,2019-01-22,2019-01-22,US,4,So far so good I was apprehensive about doing ...,"Barbara, we are so happy to hear about the out..."
15127,2020-07-31,2020-07-31,2020-07-31,US,5,Friendly. Professional. Awesome! nan,"Thank you, Keith, for your amazing feedback! O..."
23658,2018-11-08,2018-11-08,2018-11-08,US,5,Always ready to help you👏👏👏 nan,"Thank you, Jaime, for your amazing feedback! W..."


In [ ]:
# Filter the DataFrame based on 'updated' text in the 'Review Date' column
updated_reviews = df[df['Review Date'].str.contains('updated', case=False, na=False)].copy()

# Create a new column 'Updated_Date' with binary values
updated_reviews['Updated_Date'] = updated_reviews['Review Date'].str.contains('updated', case=False, na=False).astype(int)

# Get the number of data points containing 'updated'
num_updated_reviews = len(updated_reviews)
print(f"Number of data points containing 'updated': {num_updated_reviews}")

# Display the data points with 'updated' text in the 'Review Date' column and the new 'Updated_Date' column
print(updated_reviews[['Review Date', 'Updated_Date']])


Number of data points containing 'updated': 698
                Review Date  Updated_Date
9       Updated Dec 7, 2019             1
69     Updated Dec 12, 2019             1
106    Updated Nov 26, 2019             1
109    Updated Nov 26, 2019             1
209     Updated Nov 2, 2020             1
...                     ...           ...
38837   Updated Oct 4, 2020             1
38895   Updated Nov 5, 2020             1
38909  Updated Jun 17, 2021             1
38921  Updated Sep 20, 2020             1
39052  Updated Aug 31, 2020             1

[698 rows x 2 columns]


In [ ]:
df.head()

In [ ]:
# Filter the DataFrame based on 'updated' text in the 'Review Date' column
updated_reviews = df[df['Review Date'].str.contains('updated', case=False, na=False)]

# Get the number of data points containing 'updated'
num_updated_reviews = len(updated_reviews)
print(f"Number of data points containing 'updated': {num_updated_reviews}")

# Display the data points with 'updated' text in the 'Review Date' column
print(updated_reviews)

Number of data points containing 'updated': 698
              Reviewer Name Reviews Count Country Code     Experience Date  \
9             Jacey Roncali      3reviews           US   December 03, 2019   
69                    Emily      3reviews           US   November 28, 2019   
106           James Perkins      4reviews           US   November 26, 2019   
109             Wilda Valle       1review           US   November 26, 2019   
209            Pat Kilbourn      2reviews           US   November 22, 2019   
...                     ...           ...          ...                 ...   
38837  Oscar Bernales Simon       1review           US  September 29, 2020   
38895       Victoria Watson      3reviews           US  September 24, 2020   
38909                Sylvia      2reviews           US  September 22, 2020   
38921              Mai Vang      2reviews           AU  September 20, 2020   
39052       bridget swenson       1review           US     August 31, 2020   

       Rating  

In [ ]:
# Remove "review" and "reviews" from the values in the "Reviews Count" column
df['Reviews Count'] = df['Reviews Count'].str.replace('reviews', '').str.replace('review', '')

# Convert the "Reply Date" column to a string data type
df['Reply Date'] = df['Reply Date'].astype(str)

# Remove "Reply from Freedom Debt Relief" from the "Reply Date" column in the existing DataFrame
df['Reply Date'] = df['Reply Date'].str.replace("Reply from Freedom Debt Relief", "")

# Convert 'Experience Date', 'Review Date', 'Reply Date' columns to date
df[['Experience Date', 'Review Date', 'Reply Date']] = df[['Experience Date', 'Review Date', 'Reply Date']].apply(pd.to_datetime, errors='coerce')

# Convert "Reviews Count" column to integer
df['Reviews Count'] = pd.to_numeric(df['Reviews Count'], errors='coerce').astype('Int64')

In [ ]:
# Missing values
missing_values = df.isnull().sum()
print(missing_values)

In [ ]:
# Drop rows with missing values in specific columns
df.dropna(subset=['Reviewer Name', 'Experience Date', 'Rating', 'Review Date', 'Review Title', 'Review Text'], inplace=True)

In [ ]:
# Add a new column with country names
df['Country Name'] = df['Country Code'].apply(lambda code: pycountry.countries.get(alpha_2=code).name if pycountry.countries.get(alpha_2=code) else None)

# Replace the existing 'Country Name' column with the updated one
df['Country Name'] = df['Country Name']

# Display the updated DataFrame
df.head().T

In [ ]:
df.info()

In [ ]:
# Save the DataFrame to a CSV file
df.to_csv('cleaned1.csv', index=False)

**Unique Values**

In [ ]:
df.nunique()

## RATING ANALYSIS BY COUNTRY

In [ ]:
# Number of unique countries
number_of_countries = df['Country Name'].nunique()
print("Number of unique countries:", number_of_countries)

In [ ]:
# Extract unique country codes and their counts
unique_country_codes = df['Country Code'].unique()
code_counts = df['Country Code'].value_counts().reset_index()
code_counts.columns = ['Country Code', 'Number of Codes']

# Create a new DataFrame with the unique country codes and their counts
new_df = pd.DataFrame({'Country Code': unique_country_codes})
new_df = new_df.merge(code_counts, on='Country Code', how='left')

# Sort the new DataFrame in descending order based on the 'Number of Codes' column
new_df = new_df.sort_values(by='Number of Codes', ascending=False)

# Reset the index
new_df = new_df.reset_index(drop=True)

# Display the first 10 rows with country names
first_10_copy = new_df.head(10).copy()  # Ensure you create a copy

# Map country codes to names using pycountry
first_10_copy.loc[:, 'Country Name'] = first_10_copy['Country Code'].apply(lambda code: pycountry.countries.get(alpha_2=code).name)

print(first_10_copy[['Country Code', 'Number of Codes', 'Country Name']])

In [ ]:
# Calculate the mean ratings
mean_ratings = df.groupby('Country Code')['Rating'].mean().reset_index()
mean_ratings.columns = ['Country Code', 'Mean Rating']

# Merge the mean ratings with the first_10_copy DataFrame
first_10_copy = first_10_copy.merge(mean_ratings, on='Country Code', how='left')

# If 'first_10_copy' is missing the 'Mean Rating' column, make sure to recreate it
if 'Mean Rating' not in first_10_copy.columns:
    first_10_copy['Mean Rating'] = mean_ratings['Mean Rating']

print(first_10_copy[['Country Code', 'Number of Codes', 'Country Name', 'Mean Rating']])

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Calculate the mean ratings
mean_ratings = df.groupby('Country Code')['Rating'].mean().reset_index()
mean_ratings.columns = ['Country Code', 'Mean Rating']

# Merge the mean ratings with the first_10_copy DataFrame
first_10_copy = first_10_copy.merge(mean_ratings, on='Country Code', how='left', suffixes=('_original', '_mean'))

# If 'first_10_copy' is missing the 'Mean Rating' column, make sure to recreate it
if 'Mean Rating' not in first_10_copy.columns:
    first_10_copy['Mean Rating'] = mean_ratings['Mean Rating']

# Display the DataFrame with mean ratings
print(first_10_copy[['Country Code', 'Number of Codes', 'Country Name', 'Mean Rating']])

# Bar plot for Mean Ratings by Country
plt.figure(figsize=(12, 8))
ax = sns.barplot(x='Country Name', y='Mean Rating_mean', data=first_10_copy, palette='viridis')
plt.title('Mean Ratings by First 10 Country', fontsize=24, fontweight='bold', color='red')
plt.xlabel('', fontsize=14)
plt.ylabel('Mean Rating', fontsize=18, fontweight='bold', color='red')
plt.xticks(rotation=90, fontsize=18, fontweight='bold', color='red')

# Add labels to the bars at the center of each bar
for i, val in enumerate(first_10_copy['Mean Rating_mean']):
    ax.text(i, val/2, f'{val:.2f}', ha='center', va='center', fontsize=22, fontweight='bold', color='white', rotation=90)

plt.show()

In [ ]:
# Assuming you have a DataFrame named df with a 'Review Date' column and a 'Rating' column

# Extract the year from the 'Review Date' column
df['Year'] = df['Review Date'].dt.year

# Count the occurrences of each rating for each year
yearly_rating_counts = df.groupby(['Year', 'Rating']).size().unstack(fill_value=0)

# Create a figure with separate subplots for each year
fig, axes = plt.subplots(3, 4, figsize=(16, 12))

# Flatten the axes array for easier iteration
axes = axes.flatten()

# Iterate over each year and create a pie chart
for ax, (year, ratings) in zip(axes, yearly_rating_counts.iterrows()):
    colors = plt.cm.Set2(np.arange(1, len(ratings) + 1))  # Change color palette to Set2

    # Change the color of the "5" rating to green
    colors = ['green' if rating == 5 else color for rating, color in zip(ratings.index, colors)]

    explode = tuple(0.1 if i == ratings.idxmax() else 0 for i in range(1, len(ratings) + 1))

    # Check if ratings are not all zeros
    if not ratings.eq(0).all():
        # Adjust autopct to show percentages as numbers
        ax.pie(ratings, labels=None, autopct=lambda p: f'{p:.1f}', startangle=90, colors=colors, explode=explode, pctdistance=0.85, textprops={'fontsize': 10, 'weight': 'bold'})
        ax.set_title(f'{year}', fontsize=14, weight='bold')
        ax.axis('equal')  # Equal aspect ratio ensures that pie is drawn as a circle.
    else:
        ax.axis('off')  # Turn off axis for empty subplots

# Create a legend at the lower right with a centered layout
legend = fig.legend(ratings.index, title='Ratings', loc='lower right', bbox_to_anchor=(0.9, 0.1), fontsize=12, title_fontsize=12, edgecolor='black', ncol=len(ratings.index))
legend.get_title().set_fontweight('bold')  # Set fontweight for legend title

# Add a main title above the subplots
plt.figtext(0.5, 0.96, 'Yearly Ratings Percentage Distribution', ha='center', va='center', fontsize=20, fontweight='bold', color='red')

# Remove empty subplots from the layout
fig.delaxes(axes[-1])
fig.delaxes(axes[-2])

# Adjust layout for better spacing
plt.tight_layout(rect=[0, 0, 1, 0.92])  # Adjust the rect parameter to leave space for the main title

# Show the pie charts
plt.show()


## Aggregation and Grouping (yearly, quarterly, monthly):


# Yearly

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Downsample the data by averaging ratings by year
yearly_ratings = df.resample('Y', on='Review Date')['Rating'].mean().dropna()

# Create a time series plot of downsampled ratings
plt.figure(figsize=(12, 6))

# Plot the line
plt.plot(yearly_ratings.index, yearly_ratings.values, marker='o', linestyle='-', color='b', label='Average Rating')

# Fill the area under the line with light purple
plt.fill_between(yearly_ratings.index, yearly_ratings.values, color='lavender')

# Set the y-axis range to 4 to 5
plt.ylim(4.35, 4.7)

# Add labels and title
plt.title('Yearly Average Ratings Over Time', fontsize=18, fontweight='bold', color='red')
plt.xlabel('')
plt.ylabel('Average Rating', fontsize=14, fontweight='bold', color='red')
plt.grid(True)

# Rotate x-axis labels by 90 degrees
plt.xticks(rotation=90)

# Make y-axis labels bold
plt.yticks(fontweight='bold', color='blue')
plt.xticks(fontweight='bold', fontsize=13, rotation=45, color='blue')

# Annotate data values on the line plot
for x, y in zip(yearly_ratings.index, yearly_ratings.values):
    plt.text(x, y, f'{y:.2f}', ha='left', va='bottom', fontsize=14, fontweight='bold', color='green', rotation=30)

# Show the plot
plt.tight_layout()
plt.show()

# Convert the downsampled ratings to a DataFrame
yearly_ratings_df = yearly_ratings.to_frame().reset_index()
yearly_ratings_df.columns = ['Year', 'Average Rating']

# Save the DataFrame to a CSV file
yearly_ratings_df.to_csv('yearly_ratings.csv', index=False)


# Quarterly

In [ ]:
import matplotlib.pyplot as plt

# Group data by quarter and calculate the mean rating for each quarter
quarterly_mean_rating = df.groupby(df['Review Date'].dt.to_period('Q')).agg({'Rating': 'mean'})

# Create a line plot for quarterly mean rating with enhanced styling
plt.figure(figsize=(12, 6))

# Plot the line
plt.plot(
    quarterly_mean_rating.index.strftime('%b %Y'),
    quarterly_mean_rating['Rating'],
    marker='o',
    color='skyblue',
    linewidth=2,
    markersize=8,
)

# Fill the area below the line with light green color
plt.fill_between(
    quarterly_mean_rating.index.strftime('%b %Y'),
    quarterly_mean_rating['Rating'],
    color='lightgreen',  # Specify the fill color as light green
    alpha=0.3,  # Adjust transparency of the fill
)

# Set the y-axis range to 4 to 5
plt.ylim(4.25, 4.7)

# Add labels and titles
plt.title('Quarterly Mean Rating', fontsize=16, fontweight='bold', color='black')  # Default title color
plt.xlabel('', fontsize=14, fontweight='bold')
plt.ylabel('Mean Rating', fontsize=16, fontweight='bold')

# Rotate x-axis labels for readability (rotate 45 degrees)
plt.xticks(rotation=45, ha="right", fontsize=12)

# Annotate data values on the line plot and draw lines to x-axis
for x, y in zip(quarterly_mean_rating.index.strftime('%b %Y'), quarterly_mean_rating['Rating']):
    plt.annotate(
        f'{y:.2f}',
        (x, y),
        textcoords='offset points',
        xytext=(0, -15),  # Adjust the y-coordinate to move labels to the bottom
        ha='right',  # Rotate 45 degrees
        fontsize=9,
        color='blue',
        fontweight='bold',
        rotation=45,  # Rotate the label
    )
    plt.vlines(x, 3.5, y, colors='gray', linestyles='dotted', lw=1)

# Show the plot
plt.tight_layout()

# Set the title font properties
title = plt.title('Quarterly Mean Rating', fontsize=20, fontweight='bold', color='red')  # Updated title color

# Show the plot
plt.show()


# Monthly

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import pandas as pd

# Assuming you have a DataFrame named df with a 'Review Date' column and a 'Rating' column

# Downsample the data by averaging ratings by month
monthly_ratings = df.resample('M', on='Review Date')['Rating'].mean().dropna()

# Create a time series plot of downsampled ratings
plt.figure(figsize=(12, 6))
plt.plot(monthly_ratings.index, monthly_ratings.values, marker='o', linestyle='-', color='b', label='Average Rating')
plt.fill_between(monthly_ratings.index, monthly_ratings.values, color='lightgreen', alpha=0.3)  # Fill the area under the line with light green

# Add vertical lines between x-axis and points
for date, rating in zip(monthly_ratings.index, monthly_ratings.values):
    plt.vlines(date, 4, rating, color='gray', linestyle='--', linewidth=0.8)

plt.title('Monthly Average Ratings Over Time', fontsize=18, fontweight='bold', color='red')  # Updated title color
plt.xlabel('')
plt.ylabel('Average Rating', fontsize=14, fontweight='bold', color='red')
plt.grid(True)

# Set x-axis tick locator and formatter for years only
years = mdates.YearLocator()
years_fmt = mdates.DateFormatter('%Y')  # Display years as four-digit numbers

plt.gca().xaxis.set_major_locator(years)
plt.gca().xaxis.set_major_formatter(years_fmt)

# Customize x-axis ticks and labels
plt.xticks(fontsize=12, fontweight='bold', color='red', ha='center', rotation=45)

# Customize y-axis ticks and labels
plt.yticks(fontsize=12, fontweight='bold', color='red')

# Set the y-axis range to 4 to 5
plt.ylim(4, 5)

# Show the plot
plt.tight_layout()
plt.show()

# Convert the downsampled ratings to a DataFrame
monthly_ratings_df = monthly_ratings.to_frame().reset_index()
monthly_ratings_df.columns = ['Date', 'Average Rating']

# Save the DataFrame to a CSV file
monthly_ratings_df.to_csv('monthly_ratings.csv', index=False)


## Date-Related Calculations:




### a. Calculate Time Difference:


Calculate the time difference between two date columns. For instance, you can calculate the response time by subtracting the review date from the reply date.

In [ ]:
df['Review Response'] = df['Review Date'] - df['Experience Date']

df['Reply Response'] = df['Reply Date'] - df['Review Date']

In [ ]:
df.head()

### b. Extract Date Components:

Extract specific components from the date, such as year, month, day, or weekday.

In [ ]:
df['Year'] = df['Review Date'].dt.year
df['Month'] = df['Review Date'].dt.month
df['Day'] = df['Review Date'].dt.day
df['Weekday'] = df['Review Date'].dt.day_name()

### Ratings Over Time (Monthly Average)

In [ ]:
# Downsample the data by averaging ratings by month
monthly_ratings = df.resample('M', on='Review Date')['Rating'].mean().dropna()

# Create a time series plot of downsampled ratings
plt.figure(figsize=(12, 6))
plt.plot(monthly_ratings.index, monthly_ratings.values, marker='o', linestyle='-', color='b')
plt.title('Monthly Average Ratings Over Time', fontsize=18, fontweight='bold', color='red')  # Updated title color
plt.xlabel('Date', fontsize=14, fontweight='bold')
plt.ylabel('Average Rating', fontsize=14, fontweight='bold')
plt.grid(True)

# Set x-axis tick locator and formatter for years only
years = mdates.YearLocator()
years_fmt = mdates.DateFormatter('%Y')  # Display years as four-digit numbers

plt.gca().xaxis.set_major_locator(years)
plt.gca().xaxis.set_major_formatter(years_fmt)

# Show the plot
plt.tight_layout()
plt.show()

# Convert the downsampled ratings to a DataFrame
monthly_ratings_df = monthly_ratings.to_frame().reset_index()
monthly_ratings_df.columns = ['Date', 'Average Rating']

# Save the DataFrame to a CSV file
monthly_ratings_df.to_csv('monthly_ratings.csv', index=False)

# Display the DataFrame
print("Downsampled Monthly Ratings as DataFrame:")
print(monthly_ratings_df)


### Visualization for Mean Rating by Year and Month:

a. Calculate summary statistics for different time periods, such as the mean rating for each year or month.

In [ ]:
# Mean rating by year
mean_rating_by_year = df.groupby(df['Review Date'].dt.year)['Rating'].mean()

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd

# Assuming you have a DataFrame named df with a 'Review Date' column and a 'Rating' column

# Calculate the mean rating by year
mean_rating_by_year = df.groupby(df['Review Date'].dt.year)['Rating'].mean()

# Calculate the mean rating by month
mean_rating_by_month = df.groupby(df['Review Date'].dt.month)['Rating'].mean()

# Create subplots for both graphs
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 10))

# Plot the mean rating by year
ax1.plot(mean_rating_by_year.index, mean_rating_by_year.values, marker='o', linestyle='-', color='b', linewidth=3)  # Increase linewidth
ax1.fill_between(mean_rating_by_year.index, mean_rating_by_year.values, color='lightblue', alpha=0.3)
ax1.set_title('Mean Rating by Year', fontsize=16, color='red', fontweight='bold')
ax1.set_xlabel('')
ax1.set_ylabel('Mean Rating', fontsize=12, fontweight='bold')
ax1.grid(True)
ax1.set_ylim(4.4, 4.66)  # Set y-axis limits

# Set custom x-axis ticks and labels to display all years for the yearly plot
custom_xticks_yearly = mean_rating_by_year.index
ax1.set_xticks(custom_xticks_yearly)
ax1.set_xticklabels(custom_xticks_yearly, rotation=45, ha='right', fontsize=10, fontweight='bold', color='red')  # Rotating year labels for better readability

# Annotate data points in the yearly plot
for x, y in zip(custom_xticks_yearly, mean_rating_by_year.values):
    ax1.annotate(f'{y:.2f}', (x, y), textcoords="offset points", xytext=(0, -15), ha='center', fontsize=13, fontweight='bold', color='green')  # Adjusted fontsize

# Plot the mean rating by month
ax2.plot(mean_rating_by_month.index, mean_rating_by_month.values, marker='o', linestyle='-', color='g', linewidth=3)  # Increase linewidth
ax2.fill_between(mean_rating_by_month.index, mean_rating_by_month.values, color='lightgreen', alpha=0.3)
ax2.set_title('Mean Rating by Month', fontsize=16, color='red', fontweight='bold')
ax2.set_xlabel('')
ax2.set_ylabel('Mean Rating', fontsize=12, fontweight='bold')
ax2.grid(True)
ax2.set_ylim(4.56, 4.65)  # Set y-axis limits

# Set custom x-axis ticks and labels to display full month names for the monthly plot
custom_xticks_monthly = list(range(1, 13))  # Months 1 to 12
custom_xtick_labels_monthly = ['January', 'February', 'March', 'April', 'May', 'June', 'July', 'August', 'September', 'October', 'November', 'December']
ax2.set_xticks(custom_xticks_monthly)
ax2.set_xticklabels(custom_xtick_labels_monthly, rotation=45, ha='right', fontsize=13, fontweight='bold', color='red')  # Adjusted fontsize

# Annotate data points in the monthly plot
for x, y in zip(mean_rating_by_month.index, mean_rating_by_month.values):
    ax2.annotate(f'{y:.2f}', (x, y), textcoords="offset points", xytext=(0, -15), ha='center', fontsize=13, fontweight='bold', color='blue')  # Adjusted fontsize

# Adjust spacing between subplots
plt.tight_layout()

# Show the plots
plt.show()


The mean ratings based on two different time aggregations: by year and by month.

Let's discuss and conclude the insights from this data:

**Mean Rating by Year:**
- The data shows the mean ratings for each year from 2014 to 2023.
- In 2014, the mean rating was approximately 4.41, indicating a relatively positive sentiment in customer reviews.
- The mean rating increased in 2015 to approximately 4.54, suggesting an improvement in customer satisfaction.
- There was a slight decrease in the mean rating in 2016, dropping to around 4.42.
- Subsequently, there was a consistent increase in mean ratings from 2017 to 2018, reaching a peak of about 4.66.
- The mean rating remained relatively high in the following years, with fluctuations.
- In 2023, the mean rating reached approximately 4.58.

**Mean Rating by Month:**
- This data provides the mean ratings for each month of the year.
- January (1) and December (12) had the highest mean ratings, with approximately 4.60 and 4.65, respectively. These months might correspond to periods of higher customer satisfaction.
- April (4) and October (10) had slightly lower mean ratings, around 4.58.
- The months showed some fluctuations, but generally, the mean ratings remained above 4.57 throughout the year.

**Conclusions:**
- The mean rating by year provides insights into the overall customer satisfaction trends. While there were some fluctuations, it's evident that ratings improved from 2014 to 2018, remained relatively stable until 2022, and saw a slight increase in 2023.
- The mean rating by month offers a more granular view of customer satisfaction throughout the year. January and December had the highest ratings, possibly due to seasonal factors or holidays.
- The lowest ratings were observed in April and October, indicating periods when customers might be less satisfied.

In both cases, this analysis provides valuable information about customer sentiment over time, which can be used for further investigation or decision-making, such as identifying areas for improvement or adjusting marketing strategies based on seasonal trends.

### Time Series Plot of Ratings Over Time:



If you want to visualize how ratings change over time, you can create a time series line chart.

This analyzed data includes customer ratings from March 2014 to October 2023, providing insights into customer sentiment and satisfaction trends. The objective is to inform data-driven decision-making and enhance our services to meet customer expectations.

**Key Findings**

1. **Consistent Positive Sentiment**: The data reveals a consistent positive sentiment among customers, with average ratings ranging between 4.2 and 4.7. This indicates a high level of overall satisfaction with our services.

2. **Seasonal Variations**: Seasonal variations in ratings are observed, with some months showing higher average ratings and others lower. Identifying the factors contributing to these seasonal fluctuations can help optimize service delivery and resource allocation.

3. **Annual Trends**: Over the years, annual trends are evident, with specific periods exhibiting higher or lower average ratings. Understanding these trends can inform strategic planning and resource allocation.

4. **Data-Driven Decisions**: The dataset provides the opportunity to make data-driven decisions for service enhancement. Analyzing customer feedback and its correlation with ratings can guide improvements in specific areas of service delivery.

5. **Competitive Advantage**: Maintaining consistently high ratings positions Freedom Debt Relief Company as a market leader. Leveraging customer satisfaction can attract new clients and bolster our competitive advantage.

**Recommendations**

- Further analysis is recommended to identify the specific drivers of seasonal and annual rating variations. This analysis should include examining customer feedback for insights.
- Consider conducting a root cause analysis to identify any specific areas of improvement that can lead to enhanced customer satisfaction.

Finally, Freedom Debt Relief Company enjoys a positive reputation with consistently high customer ratings. Leveraging this reputation for continuous improvement is essential for maintaining a competitive advantage. By delving deeper into the drivers of rating variations and aligning service delivery with customer expectations, we can further solidify our position in the market.


In [ ]:
# Calculate the mean rating by year
mean_rating_by_year = df.groupby(df['Review Date'].dt.year)['Rating'].mean()

# Print the results
print("Mean Rating by Year:")
print(mean_rating_by_year)

# Create a plot for the mean rating by year
plt.figure(figsize=(12, 6))
plt.plot(mean_rating_by_year.index, mean_rating_by_year.values, marker='o', linestyle='-', color='b')
plt.title('Mean Rating by Year', fontsize=18, fontweight='bold', color='red')  # Title with 18 pt and bold
plt.xlabel('Year', fontsize=12, fontweight='bold')  # xlabel with 12 pt and bold
plt.ylabel('Mean Rating', fontsize=12, fontweight='bold')  # ylabel with 12 pt and bold
plt.grid(True)

# Set custom x-axis ticks and labels to display all years
custom_xticks_yearly = mean_rating_by_year.index
plt.xticks(custom_xticks_yearly, custom_xticks_yearly, rotation=45)  # Rotating year labels for better readability

# Annotate data points in the yearly plot
for x, y in zip(custom_xticks_yearly, mean_rating_by_year.values):
    plt.annotate(f'{y:.2f}', (x, y), textcoords="offset points", xytext=(0, 10), ha='center')

# Show the plot
plt.tight_layout()
plt.show()


In [ ]:
# Calculate the mean rating by month
mean_rating_by_month = df.groupby(df['Review Date'].dt.month)['Rating'].mean()

# Print the results
print("Mean Rating by Month:")
print(mean_rating_by_month)

# Set seaborn style for enhanced aesthetics
sns.set(style="whitegrid")

# Create a plot for the mean rating by month with enhanced styling
plt.figure(figsize=(12, 6))
plt.plot(mean_rating_by_month.index, mean_rating_by_month.values, marker='o', linestyle='-', color='g', linewidth=2)

# Title with 18 pt and bold
plt.title('Mean Rating by Month', fontsize=18, fontweight='bold', color='red')

# xlabel with 12 pt and bold
plt.xlabel('Month', fontsize=12, fontweight='bold')

# ylabel with 12 pt and bold
plt.ylabel('Mean Rating', fontsize=12, fontweight='bold')

# Set custom x-axis ticks and labels to display all 12 months
custom_xticks_monthly = list(range(1, 13))  # Months 1 to 12
custom_xtick_labels_monthly = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']
plt.xticks(custom_xticks_monthly, custom_xtick_labels_monthly)  # Custom month labels

# Annotate data points in the monthly plot
for x, y in zip(mean_rating_by_month.index, mean_rating_by_month.values):
    plt.text(x, y, f'{y:.2f}', ha='center', va='bottom', fontsize=10, color='black')

# Show the plot
plt.tight_layout()
plt.show()


### Histogram of Ratings:



A histogram will show the distribution of ratings. You can see how many ratings fall into each category

In [ ]:
# Set a Seaborn style for better aesthetics
sns.set(style="whitegrid")

# Create a bar plot of ratings
plt.figure(figsize=(8, 6))
rating_counts = df['Rating'].value_counts().sort_index()
ax = plt.bar(rating_counts.index, rating_counts, color='skyblue', edgecolor='black')

# Set custom bar labels at the center of each bar
for bar in ax:
    bar_x = bar.get_x() + bar.get_width() / 2
    bar_height = bar.get_height()
    plt.text(bar_x, bar_height, str(int(bar_height)), ha='center', va='bottom', fontsize=10, fontweight='bold')

# Add title, xlabel, and ylabel with specified font sizes and styles
plt.title('Distribution of Ratings', fontsize=18, fontweight='bold', color='red')
plt.xlabel('Rating', fontsize=14, fontweight='bold')
plt.ylabel('Count', fontsize=14, fontweight='bold')

# Show the plot
plt.tight_layout()
plt.show()

# Print information about the distribution of ratings
print("Distribution of Ratings:")
print(rating_counts)


The distribution of ratings, as provided in the histogram results, offers insights into how customers have rated a product or service.

Here is a discussion and conclusion based on the given distribution:

**Distribution of Ratings:**
- The data is divided into five rating categories: 1, 2, 3, 4, and 5.
- Each category represents the number of reviews with that particular rating.
- The highest number of reviews falls into the 5-star category, with a count of 30,639.
- The 4-star rating is the second most common, with 4,758 reviews.
- The 3-star rating has a count of 1,622.
- The 2-star rating is the least common among positive ratings, with only 694 reviews.
- The lowest rating, 1 star, has a count of 1,365.

**Discussion:**
- The data clearly shows that the majority of customers have given positive ratings, with 5-star and 4-star ratings being the most prevalent. This indicates that a substantial portion of customers are satisfied with the product or service, as these ratings are above the median score (3).
- The distribution is right-skewed, meaning there are far more positive reviews than negative ones. This skew suggests that the product or service generally meets customer expectations or even exceeds them.
- The relatively low counts in the 1-star and 2-star categories suggest that only a minority of customers expressed strong dissatisfaction.
- The 3-star category, while not as common as 4-star and 5-star ratings, still has a notable number of reviews, indicating a moderate level of satisfaction.

**Conclusion:**
The distribution of ratings reflects a generally positive sentiment among customers who have left reviews. The bulk of reviews fall into the higher rating categories, suggesting that the product or service is well-received. However, the existence of lower ratings (1 to 3 stars) indicates that there is room for improvement or that a subset of customers had less positive experiences.

To draw more specific conclusions or make further decisions based on this data, it may be helpful to conduct sentiment analysis on the reviews themselves or to analyze other factors, such as the time of the reviews or the topics mentioned in the reviews. This additional context can provide a deeper understanding of customer feedback.

### TEXT PREPROCESSING

#### Concatenate Review Title and Review Text Columns

In [ ]:
columns_to_concat = ['Review Title', 'Review Text']

# Concatenate the selected columns into a new column 'Concatenated_Text'
df['Concatenated_Text'] = df[columns_to_concat].apply(lambda x: ' '.join(x.dropna().astype(str)), axis=1)

#### Text Preprocessing

Before performing NLP tasks, it's important to preprocess the text data, which typically involves removing stopwords, punctuation, and converting text to lowercase.

In [ ]:
def preprocess_text(text):
    if isinstance(text, str):  # Check if the value is a string
        # Remove emojis from the text and store them separately
        emojis = [c for c in text if c in emoji.UNICODE_EMOJI]

        # Tokenize the text
        words = word_tokenize(text.lower())

        # Remove stopwords and punctuation
        words = [word for word in words if word.isalnum() and word not in stopwords.words('english')]

        # Lemmatize words
        lemmatizer = WordNetLemmatizer()
        words = [lemmatizer.lemmatize(word) for word in words]

        # Combine words and emojis back into a preprocessed text
        preprocessed_text = ' '.join(words) + ' '.join(emojis)
        return preprocessed_text
    else:
        return ''  # Return an empty string for non-text values (NaN)

In [ ]:
def preprocess_text(text):
    if isinstance(text, str):  # Check if the value is a string
        # Remove emojis from the text
        text = re.sub(r'[\U0001F600-\U0001F64F\U0001F300-\U0001F5FF\U0001F680-\U0001F6FF\U0001F700-\U0001F77F\U0001F780-\U0001F7FF\U0001F800-\U0001F8FF\U0001F900-\U0001F9FF\U0001FA00-\U0001FA6F\U0001FA70-\U0001FAFF\U0001FB00-\U0001FBFF\U0001FC00-\U0001FCFF\U0001FD00-\U0001FDFF\U0001FE00-\U0001FEFF\U0001FF00-\U0001FFFF]+', '', text)

        # Tokenize the text
        words = word_tokenize(text.lower())

        # Remove stopwords and punctuation
        words = [word for word in words if word.isalnum() and word not in stopwords.words('english')]

        # Lemmatize words
        lemmatizer = WordNetLemmatizer()
        words = [lemmatizer.lemmatize(word) for word in words]

        return ' '.join(words)
    else:
        return ''  # Return an empty string for non-text values (NaN)

In [ ]:
df['Cleaned_Concatenated_Text'] = df['Concatenated_Text'].apply(preprocess_text)

In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
# Assuming df is your DataFrame
df.to_csv('Freedom_Debt_Relief2.csv', index=False)